# 은전한닢 로드

https://github.com/SOMJANG/Mecab-ko-for-Google-Colab

In [ ]:
! git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git

In [ ]:
%cd Mecab-ko-for-Google-Colab

In [ ]:
!bash install_mecab-ko_on_colab190912.sh

In [ ]:
from konlpy.tag import Mecab

mecab = Mecab()

In [ ]:
mecab.morphs('동해물과 백두산이 마르고 닳도록')

# 패키지로드

In [ ]:
!pip install transformers
!pip install tensorboardx

# pyrouge

In [ ]:
!pip install pyrouge --upgrade
!pip install https://github.com/bheinzerling/pyrouge/archive/master.zip
!pip install pyrouge
!pip show pyrouge
!git clone https://github.com/andersjo/pyrouge.git
from pyrouge import Rouge155
!pyrouge_set_rouge_path 'pyrouge/tools/ROUGE-1.5.5'

In [ ]:
!sudo apt-get install libxml-parser-perl

In [ ]:
%%shell
cd pyrouge/tools/ROUGE-1.5.5/data
rm WordNet-2.0.exc.db # only if exist
cd WordNet-2.0-Exceptions
rm WordNet-2.0.exc.db # only if exist

./buildExeptionDB.pl . exc WordNet-2.0.exc.db
cd ../
ln -s WordNet-2.0-Exceptions/WordNet-2.0.exc.db WordNet-2.0.exc.db

# Load the Repo

In [ ]:
%cd /content
!git clone https://github.com/HaloKim/KorBertSum.git
%cd /content/KorBertSum

# BERT

In [ ]:
from transformers import BertTokenizer, BertModel, BertConfig

config = BertConfig.from_pretrained('bert-base-multilingual-cased')
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertModel.from_pretrained("bert-base-multilingual-cased")

# Data Package

In [ ]:
import json

with open('/content/drive/MyDrive/Colab_Notebooks/Study/DACON/235671_한국어 문서 추출요약 AI 경진대회_data/train.jsonl', 'r') as json_file:
    json_list = list(json_file)
result = []
for json_str in json_list:
    result.append(json.loads(json_str))

In [ ]:
import pandas as pd
df = pd.DataFrame(result)
df.head()

In [ ]:
test = df
test['morphs'] = 0
for i in range(len(test)):
  arr = []
  for j in test['article_original'][i]:
    arr.append(' '.join(mecab.morphs(j)))
  test['morphs'][i] = arr 

In [ ]:
test.to_csv('/content/drive/MyDrive/Colab_Notebooks/Study/BertSum-master/output.csv')

In [ ]:
import pandas as pd
import json

test = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/Study/BertSum-master/output.csv')
test.head()

In [ ]:
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(test, test_size = 0.3)
valid_set, test_set = train_test_split(test_set, test_size = 0.2)

In [ ]:
import ast
from tqdm import tqdm
import os

list_dic = []
for idx, row in train_set.iterrows():
  raw = row['morphs']
  target_idx = ast.literal_eval(row['extractive'])

  sentences = raw.split(',')
  src = [i.split(' ') for i in sentences]
  tgt = [a for i,a in enumerate(src) if i in target_idx]
  
  mydict = {}
  mydict['src'] = src
  mydict['tgt'] = tgt
  list_dic.append(mydict)
        
temp = []
for i,a in enumerate(tqdm(list_dic)):
  if (i+1)%6!=0:
      temp.append(a)
  else:
      filename = 'korean.'+'train'+'.'+str(i//6)+'.json'
      with open(os.getcwd()+'/json/'+filename, "w", encoding='utf8') as json_file:
          json.dump(temp, json_file, ensure_ascii=False)
      temp = []

In [ ]:
os.getcwd()

In [ ]:
try:
  os.mkdir(os.getcwd()+'/json/val')
except:
  pass

list_dic = []
for idx, row in valid_set.iterrows():
        raw = row['morphs']
        target_idx = ast.literal_eval(row['extractive'])

        sentences = raw.split(',')
        src = [i.split(' ') for i in sentences]
        tgt = [a for i,a in enumerate(src) if i in target_idx]
        
        mydict = {}
        mydict['src'] = src
        mydict['tgt'] = tgt
        list_dic.append(mydict)
        
temp = []
for i,a in enumerate(tqdm(list_dic)):
    
    if (i+1)%6!=0:
        temp.append(a)
    else:
        filename = 'korean.'+'valid'+'.'+str(i//6)+'.json'
        with open(os.getcwd()+'/json/val/'+filename, "w", encoding='utf8') as json_file:
            json.dump(temp, json_file, ensure_ascii=False)
        temp = []

In [ ]:
try:
  os.mkdir(os.getcwd()+'/json/test')
except:
  pass

list_dic = []
for idx, row in valid_set.iterrows():
        raw = row['morphs']
        target_idx = ast.literal_eval(row['extractive'])

        sentences = raw.split(',')
        src = [i.split(' ') for i in sentences]
        tgt = [a for i,a in enumerate(src) if i in target_idx]
        
        mydict = {}
        mydict['src'] = src
        mydict['tgt'] = tgt
        list_dic.append(mydict)
        
temp = []
for i,a in enumerate(tqdm(list_dic)):
    
    if (i+1)%6!=0:
        temp.append(a)
    else:
        filename = 'korean.'+'test'+'.'+str(i//6)+'.json'
        with open(os.getcwd()+'/json/test/'+filename, "w", encoding='utf8') as json_file:
            json.dump(temp, json_file, ensure_ascii=False)
        temp = []

In [ ]:
!python ./src/preprocess.py \
  -mode format_to_bert -raw_path ./json \
  -save_path ./bert_data \
  -log_file ./logs/log.log \
  -dataset train

In [ ]:
try:
  os.mkdir(os.getcwd()+'/bert_data/val')
except:
  pass

!python ./src/preprocess.py \
  -mode format_to_bert -raw_path ./json/val \
  -save_path ./bert_data/val \
  -log_file ./logs/val_log.log \
  -dataset valid

In [ ]:
try:
  os.mkdir(os.getcwd()+'/bert_data/test')
except:
  pass

!python ./src/preprocess.py \
  -mode format_to_bert -raw_path ./json/test \
  -save_path ./bert_data/test \
  -log_file ./logs/test_log.log \
  -dataset test

# Train

In [ ]:
!python ./src/train.py \
  -mode train -encoder classifier -dropout 0.1 \
  -bert_data_path ./bert_data/korean \
  -model_path ./models/bert_classifier \
  -lr 2e-3 -visible_gpus 0 -gpu_ranks 0 -world_size 1 -report_every 100 \
  -save_checkpoint_steps 1000 -batch_size 1000 -decay_method noam -log_file ./logs/log.log

# Validation

In [ ]:
!python ./src/train.py \
  -mode validate \
  -bert_data_path ./bert_data/val/korean \
  -model_path ./models/bert_classifier \
  -visible_gpus 0 -gpu_ranks 0 -batch_size 1000 \
  -log_file ./logs/val_log.log \
  -result_path ./results/val \
  -test_from ./models/bert_classifier/model_step_1000.pt \
  -temp_dir ./temp \

# Test

In [ ]:
!python ./src/train.py \
  -mode test \
  -bert_data_path ./bert_data/test/korean \
  -model_path ./models/bert_classifier \
  -visible_gpus 0 -gpu_ranks 0 -batch_size 1000 \
  -log_file ./logs/log.log \
  -result_path ./results \
  -test_from ./models/bert_classifier/model_step_10000.pt \
  -temp_dir ./temp \